In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all" # cell 的多行输出

# 数据标准化

In [2]:
import pandas as pd
X = pd.DataFrame({'酒精含量(%)': [50, 60, 40, 80, 100], '苹果酸含量(%)': [2, 1, 1, 3, 2]})
y = [0, 0, 0, 1, 1]
X

,酒精含量(%),苹果酸含量(%)
0,50,2
1,60,1
2,40,1
3,80,3
4,100,2


## min-max标准化

In [3]:
from sklearn.preprocessing import MinMaxScaler
X_new=MinMaxScaler().fit_transform(X)
X_new

array([[0.16666667, 0.5       ],
       [0.33333333, 0.        ],
       [0.        , 0.        ],
       [0.66666667, 1.        ],
       [1.        , 0.5       ]])

## Z-score标准化

In [4]:
from sklearn.preprocessing import StandardScaler
X_new=StandardScaler().fit_transform(X)
X_new

array([[-0.74278135,  0.26726124],
       [-0.27854301, -1.06904497],
       [-1.2070197 , -1.06904497],
       [ 0.64993368,  1.60356745],
       [ 1.57841037,  0.26726124]])

# 分箱

In [6]:
import pandas as pd
data = pd.DataFrame([[22,1],[25,1],[20,0],[35,0],[32,1],[38,0],[50,0],[46,1]], columns=['年龄', '是否违约'])
data

,年龄,是否违约
0,22,1
1,25,1
2,20,0
3,35,0
4,32,1
5,38,0
6,50,0
7,46,1


## 等宽分箱

In [8]:
# 第一个参数是待分箱的列
# 第二个参数是分箱的个数
data_cut=pd.cut(data['年龄'],3)
data_cut

0    (19.97, 30.0]
1    (19.97, 30.0]
2    (19.97, 30.0]
3     (30.0, 40.0]
4     (30.0, 40.0]
5     (30.0, 40.0]
6     (40.0, 50.0]
7     (40.0, 50.0]
Name: 年龄, dtype: category
Categories (3, interval[float64, right]): [(19.97, 30.0] < (30.0, 40.0] < (40.0, 50.0]]

# 变量筛选

In [10]:
import pandas as pd
data = pd.DataFrame([[22,1],[25,1],[20,0],[35,0],[32,1],[38,0],[50,0],[46,1]], columns=['年龄', '是否违约'])
data

,年龄,是否违约
0,22,1
1,25,1
2,20,0
3,35,0
4,32,1
5,38,0
6,50,0
7,46,1


## WOE IV

In [16]:
# 数据分箱
data_cut=pd.cut(data['年龄'],3)
# 统计总客户数
# groupby()函数根据分箱内容进行归类，用count()函数进行计数，可以获得各个分箱中的总客户数
cut_group_all=data['是否违约'].groupby(data_cut).count()
# 统计违约客户数
# 用sum()函数进行求和，因为违约客户的数字标识是1，而未违约客户的数字标识是0，所以sum()函数求和的结果就是违约客户数
cut_y=data['是否违约'].groupby(data_cut).sum()
# 统计未违约客户数
cut_n=cut_group_all-cut_y
# 汇总数据
df=pd.DataFrame()
df['总数']=cut_group_all
df['坏样本']=cut_y
df['好样本']=cut_n
# 计算好坏样本比率
df['坏样本%']=df['坏样本']/df['坏样本'].sum()
df['好样本%']=df['好样本']/df['好样本'].sum()
import numpy as np
df['WOE']=np.log(df['坏样本%']/df['好样本%'])
df['IV']=df['WOE']*(df['坏样本%']-df['好样本%'])
iv=df['IV'].sum()
df
print("iv:{}".format(iv))

,总数,坏样本,好样本,坏样本%,好样本%,WOE,IV
年龄,,,,,,,
"(19.97, 30.0]",3,2,1,0.50,0.25,0.693147,0.173287
"(30.0, 40.0]",3,1,2,0.25,0.50,-0.693147,0.173287
"(40.0, 50.0]",2,1,1,0.25,0.25,0.000000,0.000000


iv:0.34657359027997264


为提高代码复用性，写成如下:

In [18]:
def cal_iv(data,cut_num,feature,target):
    '''
    data 原始数据集
    cut_num 数据分箱步骤中分箱的个数
    feature 需要计算IV值的特征变量名称
    target 目标变量名称
    NOTE:
        WOE 中的正负无穷会被替换为0
    '''
    import pandas as pd
    import numpy as np
    # 数据分箱
    data_cut=pd.cut(data[feature],cut_num)
    # 统计各分箱好坏样本总数
    cut_group_all=data[target].groupby(data_cut).count()#总样本数
    cut_y=data[target].groupby(data_cut).sum()# y样本数
    cut_n=cut_group_all-cut_y# n样本数
    #汇总基础数据
    df=pd.DataFrame()
    df['总数']=cut_group_all
    df['坏样本']=cut_y
    df['好样本']=cut_n
    # 计算好坏样本比率
    df['坏样本%']=df['坏样本']/df['坏样本'].sum()
    df['好样本%']=df['好样本']/df['好样本'].sum()
    # 计算 WOE
    df['WOE']=np.log(df['坏样本%']/df['好样本%'])
    # 替换 WOE 中的正负无穷为0
    df=df.replace({'WOE':{np.inf:0,-np.inf:0}})
    # 计算分箱IV
    df['IV']=df['WOE']*(df['坏样本%']-df['好样本%'])
    # 汇总iv
    iv=df['IV'].sum()
    print("iv:{}".format(iv))

# 多重共线性

In [20]:
import pandas as pd
df=pd.read_excel('./data/数据.xlsx')
X=df.drop(columns='Y')
y=df['Y']

In [21]:
X.corr()

,X1,X2,X3
X1,1.000000,0.992956,-0.422788
X2,0.992956,1.000000,-0.410412
X3,-0.422788,-0.410412,1.000000


In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=[variance_inflation_factor(X.values,X.columns.get_loc(i)) for i in X.columns]
vif

[259.6430487184967, 257.6315718292196, 1.302330632715429]

因为特征变量X2是X1的2倍，所以使用X1对X2和X3回归和使用X2对X1和X3回归时所得的方差膨胀系数会很大，从上述计算结果也可以看出，前2个VIF值均大于100，暗示多重共线性十分严重，应该删掉X1或X2

In [23]:
X=df[['X1','X3']]
y=df['Y']
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif=[variance_inflation_factor(X.values,X.columns.get_loc(i)) for i in X.columns]
vif

[1.289349054516766, 1.2893490545167656]

可以看出当删除x2后，两个特征变量的方差膨胀系数都小于10，说明它们之间不存在多重共线性。

# 过采样

In [27]:
import pandas as pd
data=pd.read_excel('./data/信用卡数据.xlsx')
X=data.drop(columns='分类')
y=data['分类']

In [26]:
from collections import Counter
Counter(y)

Counter({0: 1000, 1: 100})

## 随机过采样

In [28]:
# imblearn库是Python中专门用来处理数据不均衡问题的工具库
# 随机过采样的RandomOverSampler()函数
from imblearn.over_sampling import RandomOverSampler
# random_state 无意义，使每次代码结果一致
ros=RandomOverSampler(random_state=0)
# 使用原始数据的特征变量和目标变量生成过采样数据集，并分别赋给变量X_oversampled和y_oversampled


In [29]:
Counter(y_oversampled)

Counter({0: 1000, 1: 1000})

### SMOTE法过采样

In [30]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=0)
X_oversampled,y_oversampled=ros.fit_resample(X,y)

In [31]:
Counter(y_oversampled)

Counter({0: 1000, 1: 1000})

# 欠采样

In [32]:
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(random_state=0)
X_undersampled,y_undersampled=rus.fit_resample(X,y)

In [34]:
Counter(y_undersampled)

Counter({0: 100, 1: 100})

# PCA

## 二维数据

In [1]:
import numpy as np  # 也可以通过pandas库构造DataFrame数据，效果一样
X = np.array([[1, 1], [2, 2], [3, 3]])

In [2]:
from sklearn.decomposition import PCA
# 参数n_components，即保留的成分个数，这里设置其值为1，也就是将二维数据降为一维数据。
# n_components参数不仅可以设置成降维后成分的个数，还可以设置成降维后保留的信息的百分比，例如，将其设置成0.9就是在降维后保留原特征90%的信息。
# 需要注意的是，如果将n_components参数设置成降维后成分的个数，其值不能大于min（n_samples,n_features），即样本数和特征变量数两者之中的最小数。
pca=PCA(n_components=1)
pca.fit(X)
X_transformed=pca.transform(X)
X_transformed

array([[-1.41421356],
       [ 0.        ],
       [ 1.41421356]])

In [10]:
# 获取线性组合系数
pca.components_
# 打印公式
a=pca.components_[0][0]
b=pca.components_[0][1]
print('{}*X+{}*Y'.format(a,b))

array([[0.70710678, 0.70710678]])

0.7071067811865476*X+0.7071067811865475*Y


## 三维数据

In [12]:
import pandas as pd
X = pd.DataFrame([[45, 0.8, 9120], [40, 0.12, 2600], [38, 0.09,3042], [30, 0.04, 3300], [39, 0.21, 3500]], columns=['年龄(岁)', '负债比率', '月收入(元)'])
X

,年龄(岁),负债比率,月收入(元)
0,45,0.80,9120
1,40,0.12,2600
2,38,0.09,3042
3,30,0.04,3300
4,39,0.21,3500


3个维度的特征数据的量级相差较大。如果数据中某一特征的数值很大，那么它在计算中所占的比重就会很大，PCA降维时会更看重这个特征，而忽略其他数值小的特征。因此，先对3个维度的特征数据进行标准化，这也有利于PCA降维时梯度下降法的收敛

In [13]:
# Z-score 标准化
from sklearn.preprocessing import StandardScaler
X_new=StandardScaler().fit_transform(X)
# pca
from sklearn.decomposition import PCA
# 三维数据降为二维数据
pca=PCA(n_components=2)
pca.fit(X_new)
X_transformed=pca.transform(X_new)


PCA(n_components=2)

In [14]:
pca.components_

array([[ 0.52952108,  0.61328179,  0.58608264],
       [-0.82760701,  0.22182579,  0.51561609]])

In [15]:
dim = ['年龄(岁)', '负债比率', '月收入(元)']
for i in pca.components_:
    formula = []
    for j in range(len(i)):
        formula.append(str(i[j]) + ' * ' + dim[j])
    print(" + ".join(formula))

0.529521083916554 * 年龄(岁) + 0.6132817922410683 * 负债比率 + 0.5860826434841946 * 月收入(元)
-0.8276070105929828 * 年龄(岁) + 0.2218257919336098 * 负债比率 + 0.5156160917294703 * 月收入(元)
